# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [24]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [25]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp (C),Humidity (%),Cloudiness (%),Wind Speed (m/s),Country,Date
0,0,bengkulu,-3.8004,102.2655,26.50,75,99,2.60,ID,2023-02-02
1,1,mahebourg,-20.4081,57.7000,25.16,94,75,0.00,MU,2023-02-02
2,2,kavieng,-2.5744,150.7967,28.98,70,100,2.85,PG,2023-02-02
3,3,ushuaia,-54.8000,-68.3000,12.81,62,20,8.23,AR,2023-02-02
4,4,khatanga,71.9667,102.5000,-37.02,100,56,0.90,RU,2023-02-02
...,...,...,...,...,...,...,...,...,...,...
563,563,terrace,54.5163,-128.6035,2.95,93,100,7.20,CA,2023-02-02
564,564,sholokhove,47.6945,34.0312,0.01,92,22,3.15,UA,2023-02-02
565,565,fairbanks,64.8378,-147.7164,-12.88,86,100,0.00,US,2023-02-02
566,566,bowen,-20.0167,148.2333,29.82,67,85,1.47,AU,2023-02-02


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [26]:
# %%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity (%)",
    color = "City",
    frame_width = 700,
    frame_height = 500    
)


# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity (%))

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [28]:
# Narrow down cities that fit criteria and drop any results with null values
# Criteria: Max Temp between 21 and 27 C, Wind Speed less than 4.5 m/s, and Cloudiness = 0%
filtered_df = city_data_df.loc[((city_data_df['Max Temp (C)']>21)&(city_data_df['Max Temp (C)']<27)) & (city_data_df['Wind Speed (m/s)']<4.5) & (city_data_df['Cloudiness (%)'] == 0)]

# Drop any rows with null values
filtered_df.dropna()

# Display sample data
filtered_df

,City_ID,City,Lat,Lng,Max Temp (C),Humidity (%),Cloudiness (%),Wind Speed (m/s),Country,Date
41,41,caraballeda,10.6112,-66.8522,26.04,74,0,1.03,VE,2023-02-02
193,193,tecoanapa,16.5167,-98.7500,26.03,70,0,3.24,MX,2023-02-02
403,403,key largo,25.0865,-80.4473,24.44,74,0,3.29,US,2023-02-02
413,413,quatre cocos,-20.2078,57.7625,25.06,80,0,2.06,MU,2023-02-02
484,484,marsh harbour,26.5412,-77.0636,23.69,75,0,3.04,BS,2023-02-02
513,513,gulf gate estates,27.2517,-82.5147,22.27,77,0,1.54,US,2023-02-02


### Step 3: Create a new DataFrame called `hotel_df`.

In [29]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_df.copy()
hotel_df = hotel_df[["City","Country","Lat","Lng","Humidity (%)"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity (%),Hotel Name
41,caraballeda,VE,10.6112,-66.8522,74,
193,tecoanapa,MX,16.5167,-98.7500,70,
403,key largo,US,25.0865,-80.4473,74,
413,quatre cocos,MU,-20.2078,57.7625,80,
484,marsh harbour,BS,26.5412,-77.0636,75,
513,gulf gate estates,US,27.2517,-82.5147,77,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [30]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "limit":1,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
caraballeda - nearest hotel: Hotel Belmar
tecoanapa - nearest hotel: No hotel found
key largo - nearest hotel: Key Largo Inn
quatre cocos - nearest hotel: Horisun
marsh harbour - nearest hotel: Lofty Fig Villas
gulf gate estates - nearest hotel: Hibiscus Suites


,City,Country,Lat,Lng,Humidity (%),Hotel Name
41,caraballeda,VE,10.6112,-66.8522,74,Hotel Belmar
193,tecoanapa,MX,16.5167,-98.7500,70,No hotel found
403,key largo,US,25.0865,-80.4473,74,Key Largo Inn
413,quatre cocos,MU,-20.2078,57.7625,80,Horisun
484,marsh harbour,BS,26.5412,-77.0636,75,Lofty Fig Villas
513,gulf gate estates,US,27.2517,-82.5147,77,Hibiscus Suites


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [31]:
#%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity (%)",
    color = "City",
    hover_cols=["Hotel Name","Country"],
    frame_width = 700,
    frame_height = 500    
)


# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity (%),Hotel Name,Country)